<a href="https://colab.research.google.com/github/gabrielecola/Optimization_Tecniques/blob/main/First_Second_Order_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework $\alpha$ - First and second order methods

In [ ]:
import numpy as np
from numpy.core.numeric import Inf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
import plotly
import plotly.graph_objects as go
from plotly.colors import qualitative
from plotly.subplots import make_subplots

## Homework $\alpha$.1

Let $\boldsymbol{w} \in \mathbb{R}^2$. Consider the *Beale function* defined as
$$b(\boldsymbol{w}) = \left(1.5 - w^{(0)} + w^{(0)} w^{(1)}\right)^2 + \left(2.25 - w^{(0)} + w^{(0)} \ [w^{(1)}]^2\right)^2 + \left(2.625 - w^{(0)} + w^{(0)} \ [w^{(1)}]^3\right)^2.$$

This homework is inspired by [this GIF](https://i.imgur.com/pD0hWu5.gif), which is widely use on the data science websites to explain how different methods perform. Note that in the legend the red line (*SGD*) actually corresponds to a gradient method, the green line (*momentum*) corresponds to the method we called heavy ball, while the purple line (*NAG*) is associated to Nesterov method.

1. Draw a contour plot of the function $b$ on the square domain $[-4.5, 4.5]^2$. Discuss whether or not to use a logarithmic scale.

In [ ]:
domain_component_0 = [-4.5, 4.5]
domain_component_1 = [-4.5, 4.5]

In [ ]:
w_component_0 = np.linspace(domain_component_0[0], domain_component_0[1], 100)
w_component_1 = np.linspace(domain_component_1[0], domain_component_1[1], 100)

In [ ]:
f_w = np.zeros((len(w_component_0), len(w_component_1)))
for i in range(f_w.shape[0]):
    for j in range(f_w.shape[1]):
        f_w[j, i] = (
            (1.5 - w_component_0[i] + w_component_0[i] * w_component_1[j])**2
            + (2.25 - w_component_0[i] + w_component_0[i] * (w_component_1[j]**2))**2
            + (2.625 - w_component_0[i] + w_component_0[i] * (w_component_1[j]**3))**2
        )

In [ ]:
def f_1(w):
  return (1.5 - w[0] + w[0] * w[1])**2 + (2.25 - w[0] + w[0] * (w[1]**2))**2 + (2.625 - w[0] + w[0] * (w[1]**3))**2

In [ ]:
fig = go.Figure(data=[go.Surface(x=w_component_0, y=w_component_1, z=f_w)])
fig.update_layout(title="Beale function - surface plot")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Contour(x=w_component_0, y=w_component_1, z=f_w)])
fig.update_layout(title="Beale function - contour plot", width=512, height=512, autosize=False)
fig.show()

Because the function covers a wide range of magnitudes, using a logarithmic scale is more appropriate. As seen in the plot below, this new scale makes the minimum points more apparent.

In [ ]:
fig = go.Figure(data=[go.Contour(x=w_component_0, y=w_component_1, z=np.log(f_w))])
fig.update_layout(title="Beale function - contour plot on logarithmic scale", width=512, height=512, autosize=False)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(x=w_component_0, y=w_component_1, z=np.log(f_w))])
fig.update_traces(contours_z=dict(show=True, project_z=True, usecolormap=True))
fig.update_layout(title="Beale function - surface and countour plots in the same figure on logarithmic scale")
fig.show()

2. Compute the gradient $\nabla b$ and the hessian $\nabla^2 b$. Implement two Python functions for their evaluation. Use the implemented functions to verify that $\boldsymbol{w}^* = (3, 0.5)$ is a global minimum of $b$.

 *Hint*: feel free to use `sympy` to help in the computation of the partial derivatives.

The gradient $\Delta b$ is given by:

$$ \begin{align} \frac{db(\boldsymbol{w})}{dw^{(0)}} = 6w^{(0)} - 2w^{(0)}[w^{(1)}]^2 - 12.75 + 3w^{(1)} - 4 w^{(0)} w^{(1)} + 2 w^{(0)} [w^{(1)}]^4 + 4.5 [w^{(1)}]^2+2 w^{(0)} [w^{(1)}]^6 + 5.25 [w^{(1)}]^3  + - 4 w^{(0)} [w^{(1)}]^3 \end{align}$$

$$ \begin{align} \frac{db(\boldsymbol{w})}{dw^{(1)}} =
- 2w^{(1)}[w^{(0)}]^2 + 3w^{(0)} - 2 [w^{(0)}]^2 + 4 [w^{(0)}]^2 [w^{(1)}]^3 + 9 w^{(0)} w^{(1)}+ 6 [w^{(0)}]^2 [w^{(1)}]^5 + 15.75 w^{(0)} [w^{(1)}]^2  - 6 [w^{(0)}]^2 [w^{(1)}]^2 \end{align}
$$


In [ ]:
def grad_f_1(w):
    return np.array([
        (6 * w[0] - 2 * w[0] * w[1]**2 - 12.75 + 3 * w[1] - 4 * w[0] * w[1] + 2 * w[0] * w[1]**4 + 4.5 * w[1]**2 + 2 * w[0] * w[1]**6 + 5.25 * w[1]**3 - 4 * w[0] * w[1]**3),
        (- 2 * w[1] *w[0]**2 + 3 * w[0] - 2 * w[0]**2  + 4 * w[0]**2 * w[1]**3 + 9 * w[0] * w[1] + 6 * w[0]**2 * w[1]**5 + 15.75 * w[0] * w[1]**2 - 6 * w[0]**2 * w[1]**2)

    ])

Then I can check that w* = [3, 0.5] is a stationary point evaluating it within the gradient's functions.

In [ ]:
w_star = [3.0, 0.5]
grad_f_1(w_star)

array([0., 0.])

The Hessian $\Delta^2 b$ is given by:

$$ \begin{align} \frac{d^2b(\boldsymbol{w})}{d[w^{(0)}]^2} = 2[w^{(1)}]^6 + 2 [w^{(1)}]^4 -4 [w^{(1)}]^3 - 2[w^{(1)}]^2 - 4[w^{(1)}] + 6  \end{align}$$

$$ \begin{align} \frac{d^2b(\boldsymbol{w})}{d[w^{(1)}]^2} =  -2[w^{(0)}]^2 + 12 [w^{(0)}]^2 [w^{(1)}]^2 + 9 [w^{(0)}] + 30 [w^{(0)}]^2 [w^{(1)}]^4 +31.5 w^{(0)} w^{(y)} - 12 [w^{(0)}]^2 w^{(1)} \end{align}$$


$$ \begin{align} \frac{d^2b(\boldsymbol{w})}{dw^{(0)}w^{(1)}} = -4 w^{(0)} w^{(1)} + 3 - 4 w^{(0)} + 8 [w^{(0)}] [w^{(1)}]^3 + 9 w^{(1)} + 12 w^{(0)} [w^{(1)}]^5 + 15.75 [w^{(1)}]^2 - 12 w^{(0)} [w^{(1)}]^2 \end{align}$$


In [ ]:
def db_dw0w0(w0, w1):
  return 2 * w1 ** 6 + 2 * w1 ** 4 - 4 * w1 ** 3 - 2 * w1 ** 2 - 4 * w1 + 6

def db_dw0w1(w0, w1):
  return -4 * w1 * w0 + 3 - 4 * w0 + 8 * w0 * w1**3 + 9 * w1 + 12 * w0 * w1**5 + 15.75 * w1**2 - 12 * w0 * w1 ** 2

def db_dw1w1(w0, w1):
  return -2 * w0 ** 2 + 12 * w0**2 * w1 ** 2 + 9 * w0 + 30 * w0 ** 2 * w1 ** 4 + 31.5 * w0 * w1 - 12 * w0 ** 2 * w1

Then I can check that **w*** is a minimum by evaluating it into the Hessian and computing its eigenvalues

In [ ]:
w0= 3
w1= 0.5
hessian_f = np.array([[db_dw0w0(w0, w1), db_dw0w1(w0, w1)], [db_dw0w1(w0, w1), db_dw1w1(w0, w1)]])
hessian_f

array([[  3.15625, -11.4375 ],
       [-11.4375 ,  46.125  ]])

In [ ]:
eigs, _ = np.linalg.eig(hessian_f)
eigs

array([ 0.30146365, 48.97978635])

In [ ]:
eigs > 0

array([ True,  True])

Since all the **eigenvalues** are **positive** and **real**, the Hessian is positive definite and **w*** = [3, 0.5] is a minimum of the function b.

3. Implement a Python function for the minimization of a function $f$ using one of the following methods:
   * gradient descent, or
   * heavy ball method, or
   * Nesterov accelerated gradient, or
   * Adagrad, or
   * RMSProp, or
   * Adam

   Required inputs:
   1. the method to use (`gradient`, `heavy_ball`, `nesterov`, `adagrad`, `rmsprop`, `adam`)
   2. the function $f$,
   3. its gradient $\nabla f$,
   4. the (constant) coefficients appearing in the definition of the method stored in a dictionary (e.g. `{"alpha": 0.1}`), namely
       * gradient descent: the (constant) step length $\alpha$;
       * heavy ball method: the (constant) step length $\alpha$ and the (constant) momentum coefficient $\beta$;
       * Nesterov accelerated gradient: the (constant) step length $\alpha$ and the (constant) momentum coefficient $\beta$;
       * Adagrad: the (constant) step length factor $\alpha$ and the regularization coefficient $\zeta$;
       * RMSProp: the (constant) step length factor $\alpha$, the regularization coefficient $\zeta$ and the decay coefficient $\gamma$;
       * Adam: the (constant) step length factor $\alpha$, the regularization coefficient $\zeta$ and the decay coefficients $\gamma_m$ and $\gamma_s$,
   5. the tolerance $\varepsilon$ for stopping criterion on the norm of the gradient,
   6. the maximum number $K_{\max}$ of iterations, and
   7. the initial condition $\boldsymbol{w}_{0}$.
  
   Required outputs:
   1. the optimization variable iterations $\{\boldsymbol{w}_k\}_k$,
   2. the corresponding function values $\{f(\boldsymbol{w}_k)\}_k$, and
   3. the corresponding gradients $\{\nabla f(\boldsymbol{w}_k)\}_k$.

   *Hint*: you do not need to implement all the required optimization methods before moving on to the next steps of this homework. You may start with the implementation of the gradient method, which is used at the next step 4. Once you are done with step 4, come back to step 3 for the implementation of heavy ball method which is required in step 5. You may proceed in a similar way for the remaining optimization methods.

In [ ]:
def optimization(method, f, grad_f, epsilon, k_max, w_0, coef):
    assert method in ("gradient", "heavy_ball", "nesterov", "adagrad", "rmsprop", "adam")

    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]
    sHAT_k = [] #needed for "rmsprop" and "adam"
    mHAT_k = [] #needed for "adam"

    # Prepare iteration counter
    k = 0

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]
        f_k = all_f[k]
        grad_f_k = all_grad_f[k]
        grad_f_k_beta = all_grad_f[k]

        if method == "gradient":

            #Get coefficients
            alpha_k = coef.get("alpha_gd")

            # Compute w_{k+1}
            w_k_plus_1 = w_k - alpha_k * grad_f_k

        elif method == "heavy_ball":
            # Get coefficients
            alpha_k = coef.get("alpha_hb")
            beta_k = coef.get("beta_hb")

            # Compute w_{k-1}
            if k > 0:
              w_k_minus_1 = all_w[k-1]
            else:
              w_k_minus_1 = w_0

            # Compute w_{k+1}
            w_k_plus_1 = w_k - alpha_k * grad_f_k + beta_k * (w_k - w_k_minus_1)

        elif method == "nesterov":
          # Get coefficients
          alpha_k = coef.get("alpha_n")
          beta_k = coef.get("beta_n")

          # Compute w_{k-1}
          if k > 0:
            w_k_minus_1 = all_w[k-1]
          else:
            w_k_minus_1 = w_0

          # Compute w_{k+1}
          z_k = w_k + beta_k*(w_k - w_k_minus_1)
          w_k_plus_1 = w_k - alpha_k * grad_f(z_k) + beta_k * (w_k - w_k_minus_1)


        elif method == "adagrad":
          # Get coefficients
          alpha_k = coef.get("alpha_adag")
          zeta_k = coef.get("zeta_adag")

          # Compute h_k
          s_k = sum(i*i for i in all_grad_f)
          h_k = 1/(np.sqrt(s_k)+zeta_k)

          # Compute w_{k+1}
          w_k_plus_1 = w_k + alpha_k*h_k*(-grad_f_k)


        elif method == "rmsprop":
          # Get coefficients
          alpha_k = coef.get("alpha_RMS")
          zeta_k = coef.get("zeta_RMS")
          gamma_k = coef.get("gamma_RMS")

          # Compute w_{k-1}
          if k > 1:
            sHAT_k_minus_1 = sHAT_k[k-1]
          else:
            sHAT_k_minus_1 = 0

          # Compute g_k
          g_k = - grad_f_k

          # Compute s_HAT(k)
          sHAT_k.append((gamma_k * sHAT_k_minus_1) + ((1-gamma_k) * g_k) * g_k)

          # Compute w_{k+1}
          w_k_plus_1 = w_k + (alpha_k/(np.sqrt(np.array(sHAT_k[k])) + zeta_k)) * g_k


        elif method == "adam":
          #Get coefficients
          gammaM_k = coef.get("gammaM_adam")
          gammaS_k = coef.get("gammaS_adam")
          alpha_k = coef.get("alpha_adam")
          zeta_k = coef.get("zeta_adam")

          # Compute g_k
          g_k = - grad_f_k

          # Compute s_HAT_{k-1} and m_HAT_{k-1}
          if k > 1:
            mHAT_k_minus_1 = mHAT_k[k-1]
          else:
            mHAT_k_minus_1 = 0

          if k > 1:
            sHAT_k_minus_1 = sHAT_k[k-1]
          else:
            sHAT_k_minus_1 = 0

          # Compute s_HAT_{k} and m_HAT_{k}
          mHAT_k.append((gammaM_k * mHAT_k_minus_1) + (1- gammaM_k)* g_k)
          sHAT_k.append(gammaS_k* sHAT_k_minus_1 + ((1- gammaS_k)* g_k) * g_k)

          # Compute s_TILDE_{k} and m_TILDE_{k}
          mTILDE_k = mHAT_k[k]/(1- (gammaM_k**(k+1)))
          sTILDE_k = sHAT_k[k]/(1- (gammaS_k**(k+1)))

          # Compute w_{k+1}
          w_k_plus_1 = w_k + (alpha_k/(np.sqrt(np.array(sTILDE_k)) + zeta_k)) * mTILDE_k



        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))


        # Increment iteration counter
        k += 1

        if k >= k_max:
          #print("WARNING: " + method + " method terminated early due to reaching k_max = " + str(k))
          return np.array(all_w), np.array(all_f), np.array(all_grad_f), k


    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f), k

In [ ]:
# All coefficients to define for each method
coef = {"alpha_gd": 1,
        "alpha_hb": 1,
        "beta_hb": 1,
        "alpha_n": 1,
        "beta_n": 1,
        "zeta_adag": 1,
        "alpha_adag": 1,
        "gamma_RMS": 1,
        "alpha_RMS": 1,
        "zeta_RMS": 1,
        "gammaM_adam": 1,
        "gammaS_adam": 1,
        "alpha_adam": 1,
        "zeta_adam": 1}

4. Run the gradient descent method with initial point $\boldsymbol{w}_0 = (0.8, 1.6)$ and tolerance $\varepsilon = 10^{-5}$.

   The hyperparameter $\alpha$ needs to be tuned, because we do not know whether $b$ is smooth or what is its smoothness constant. Peform the following tuning: fix $K_{\max} = 100$ and choose $\alpha$ among the five choices $\{10^0, 10^{-1}, 10^{-2}, 10^{-3}, 10^{-4}\}$ accepting the choice that gives the lowest cost function at the final iteration.

   Discuss why the choice $\alpha = 10^{-2}$ outperforms the others. Feel free to use e.g.
   * tables, for instance a table with $K_{\max}$ rows and 5 columns, to compare how the cost function at the $k$ iteration changes for different values of $\alpha$, or
   * `plotly` animations, for instance an animation with $K_{\max}$ frames, where each frame contains one curve per each value of $\alpha$ showing the path of the optimization variable path until the $k$-th iteration.

   *Take-home message*: in real problems you might have to tune the hyperparameters of the optimization algorithm, and comparing different values is the simplest hyperparameter tuning experience.

In [ ]:
warnings.filterwarnings('ignore')
result = pd.DataFrame(columns=[0,1,2,3,4], index=["result", "iterations", "cost"])
for i in range(0, 5):
  coef = {"alpha_gd": 10**(-i)}
  all_w_g, all_f_g, all_grad_g,k_g = optimization(
    "gradient",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)

  if type(all_w_g[-1, 0]) != Inf and type(all_w_g[-1, 1]) != Inf :
    if all_w_g[-1, 0] > 10^100 or all_w_g[-1, 1] > 10^100: #in case of divergence
      result[i].iloc[0] = "[" + "{:.2e}".format(all_w_g[-1, 0]) + ", " + "{:.2e}".format(all_w_g[-1, 1]) + "]"
    else:
      result[i].iloc[0] = "[" + "{:.4f}".format(all_w_g[-1, 0]) + ", " + "{:.4f}".format(all_w_g[-1, 1]) + "]"

  else:
    result[i].iloc[0] = "[" +str(all_w_g[-1, 0])+ "]"
  result[i].iloc[1] = k_g
  result[i].iloc[2] = abs(all_f_g[-1] - f_1([3, 0.5]))

result.rename({0:'10^(0)', 1:'10^(-1)', 2:'10^(-2)', 3:'10^(-3)', 4:'10^(-4)'}, axis=1, inplace=True)

In [ ]:
g = result.style.set_caption("GRADIENT DESCENT: Final convergence result, iterations and cost function for different values of alpha") \
.highlight_min(axis=1, subset= "10^(-2)", props='color:black; background-color: #8deeee;')\
.format(na_rep='\\', thousands=" ")

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
g.set_table_styles([headers, cells])

,10^(0),10^(-1),10^(-2),10^(-3),10^(-4)
result,"[inf, inf]","[1.78e+243, 2.88e+243]","[2.4408, 0.3217]","[1.0486, 0.3166]","[0.6195, 1.1776]"
iterations,4,4,100,100,100
cost,\,inf,0.098490,4.911809,17.892408


As observed in the chart above, none of the methods converge within 100 iterations. However, it is evident that the function does not converge for $\alpha$ = 0 or $\alpha$ = 0.1, while it achieves some results with other values of $\alpha$. With α = $10^{(-2)}$, the function attains the lowest cost, equal to 0.09849. Therefore, α = $10^{(-2)}$ is the best option. Additional verification can be done using the charts and plot below:

In [ ]:
COST_1 = pd.DataFrame(columns=[0,1])
COST_2 = pd.DataFrame(columns=[2,3,4])

ALL_W_1 = pd.DataFrame(columns=[0,1,2,3])
ALL_W_2 = pd.DataFrame(columns=[2,3,4,5,6,7])

for i in range(0, 10, 2):
  coef = {"alpha_gd": 10**(-i//2)}
  all_w, all_f, all_grad,k = optimization(
    "gradient",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)

  if i <= 2:
    ALL_W_1.iloc[:,i]= all_w[:, 0]
    ALL_W_1.iloc[:,i+1]= all_w[:, 1]

    COST_1.iloc[:,i//2]= all_f - f_1([3, 0.5])

  else:
    COST_2.iloc[:,((i-2)//2 -1)]= all_f - f_1([3, 0.5])

    ALL_W_2.iloc[:,i-4]= all_w[:, 0]
    ALL_W_2.iloc[:,i-3]= all_w[:, 1]

COST_DF = pd.concat([COST_1, COST_2], axis =1)
COST_DF.rename({0:'10^(0)', 1:'10^(-1)', 2:'10^(-2)', 3:'10^(-3)', 4:'10^(-4)'}, axis=1, inplace=True)

In [ ]:
g1 = COST_DF.style.highlight_min(axis=1, subset= "10^(-2)", props='color:black; background-color: #8deeee;')\
.set_caption("GRADIENT DESCENT: Cost function at each interation for different values of alpha") \
.format(na_rep='\\',  formatter= {('10^(0)'): "{:.4e}", ("10^(-1)"): "{:.4e}", ('10^(-2)'): "{:.4f}", ("10^(-3)"): "{:.4f}", ("10^(-4)"): "{:.4f}"
                          })

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
g1.set_table_styles([headers, cells])

,10^(0),10^(-1),10^(-2),10^(-3),10^(-4)
0,4.2185e+01,4.2185e+01,42.1848,42.1848,42.1848
1,5.9812e+14,1.3597e+06,12.3306,34.2783,41.2938
2,5.1100e+108,1.5330e+40,11.6554,29.5513,40.4495
3,inf,3.7131e+278,10.8879,26.3770,39.6485
4,\,inf,10.0501,24.0861,38.8875
5,\,\,9.1739,22.3483,38.1635
6,\,\,8.2937,20.9808,37.4740
7,\,\,7.4400,19.8736,36.8166
8,\,\,6.6360,18.9564,36.1890
9,\,\,5.8961,18.1822,35.5893


In [ ]:
ALL_W_DF = pd.concat([ALL_W_1, ALL_W_2], axis =1)
ALL_W_DF.reset_index(drop=True, inplace=True)
a = ALL_W_DF.values.tolist()
cols = pd.MultiIndex.from_tuples([("10^(0)", "w_0"),
                                  ("10^(0)", "w_1"),
                                  ("10^(-1)", "w_0"),
                                  ("10^(-1)", "w_1"),
                                  ("10^(-2)", "w_0"),
                                  ("10^(-2)", "w_1"),
                                  ("10^(-3)", "w_0"),
                                  ("10^(-3)", "w_1"),
                                  ("10^(-4)", "w_0"),
                                  ("10^(-4)", "w_1")])
ALL_W_DF = pd.DataFrame(a,columns=cols)

g2 = ALL_W_DF.style.set_caption("GRADIENT DESCENT: Convergence results at each interation for different values of alpha") \
.format(na_rep='\\',  formatter= {('10^(0)', "w_0"): "{:.4e}",('10^(0)', "w_1"): "{:.4e}", ('10^(-1)', "w_0"): "{:.4e}", ('10^(-1)', "w_1"): "{:.4e}",("10^(-1)"): "{:.4e}"
                          })

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
g2.set_table_styles([headers, cells])

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.array(range(3)), y=COST_DF.iloc[:,0], name = "alpha= 1", mode='lines+markers')
)

fig.add_trace(
    go.Scatter(x=np.array(range(101)), y=COST_DF.iloc[:,1], name = "alpha= 10^(-1)", mode='lines+markers')
)

fig.add_trace(
    go.Scatter(x=np.array(range(101)), y=COST_DF.iloc[:,2], name = "alpha= 10^(-2)", mode='lines+markers')
)

fig.add_trace(
    go.Scatter(x=np.array(range(101)), y=COST_DF.iloc[:,3], name = "alpha= 10^(-3)", mode='lines+markers')
)

fig.add_trace(
    go.Scatter(x=np.array(range(101)), y=COST_DF.iloc[:,4], name = "alpha= 10^(-4)", mode='lines+markers')
)

fig.update_layout(height=600, width=800, title_text="Cost function for different values of alpha", yaxis_range=[0,101])
fig.show()

From this plot, we can see visually that with $\alpha= 10^{-2}$ there is the fastest decrease in the cost function already at the first iteration.

Another visualization is produced also with a larger value of kmax = 1000:

In [ ]:
all_w = [None] * 5
all_f = [None] * 5
all_grad_f = [None] * 5

In [ ]:
for run in range(5):
  coef = {"alpha_gd": 10**(-int(run))}
  all_w[run], all_f[run], all_grad_f[run], k  = optimization("gradient",
                                                           f_1,
                                                           grad_f_1,
                                                           epsilon = 10**(-5),
                                                           k_max = 1000,
                                                           w_0 = np.array([0.8, 1.6]),
                                                           coef = coef)

In [ ]:
max_K = max([all_f[run].shape[0] for run in range(5)])
max_K

1001

In [ ]:
#optimum point
w_star = np.array([3, 0.5])

In [ ]:
# Add opaque contour plot
fig = go.Figure(data=[go.Contour(
    x=w_component_0, y=w_component_1, z=np.log(f_w),
    showscale=False, visible=True, opacity=0.5
)])

# Add a red cross marker to locate w*
fig.add_trace(
    go.Scatter(x=[w_star[0]], y=[w_star[1]],
               marker=dict(color="red", size=10, symbol="x"),
               mode="markers", name="w*", visible=True))

# Prepare a slider every 100 iterations
slides = []
for k in range(int(max_K / 10)):
    for run in range(5):
        # Set non uniform marker size to highlight the current iteration k
        marker_size = np.zeros((10 * k + 1, ))
        marker_size[-1] = 10
        # Add lines
        fig.add_trace(
            go.Scatter(x=all_w[run][:10 * k + 1:10, 0],
                       y=all_w[run][:10 * k + 1:10, 1],
                       visible=True,
                       marker=dict(color=plotly.colors.qualitative.Set1[run], size=marker_size),
                       line=dict(color=plotly.colors.qualitative.Set1[run], width=2),
                       mode="lines+markers",
                       name="alpha= 10^(-" + str(run) + ") at k = " + str(k*10)))

    # Add slider tick
    slide = {
        "method": "update",
        "args": [
            {"visible": [False] * (5 * int(max_K / 10) + 2)},
            {}
        ]
    }
    slide["args"][0]["visible"][0] = True
    slide["args"][0]["visible"][1] = True
    for run in range(5):
        slide["args"][0]["visible"][5 * k + run + 2] = True
    slides.append(slide)

for run in range(5):
    fig.data[run + 2].visible = True

fig.update_layout(
    title="Optimization variable iterations over contour plot",
    width=612, height=512, autosize=False,
    sliders=[dict(steps=slides)]
)
fig.update_xaxes(range=[0, 4])
fig.update_yaxes(range=[-0.5, 2])
fig.show()

This plot shows that with $\alpha = 10^{(-2)}$, the algorithm reaches the minimum point within 1000 iterations, whereas with other values of $\alpha$, the minimum point is not reached even after 1000 iterations.

5. Run the heavy ball method with initial point $\boldsymbol{w}_0 = (0.8, 1.6)$, tolerance $\varepsilon = 10^{-5}$ and the value of $\alpha$ you found was the best one at step 4.

   The hyperparameter $\beta$ needs to be tuned as well. Peform the following tuning: choose $\beta$ among the four choices $\{0.9, 0.8, 0.7, 0.6\}$, accepting the choice that gives the lowest cost function at the final iteration. Repeat this tuning for $K_{\max} = 10$ and $K_{\max} = 100$.

   Is $K_{\max} = 10$ enough to decide which value of $\beta$ to choose, or do you need to have $K_{\max} = 100$ to take the best decision? Does the selected value (which will turn out to be $\beta = 0.8$) outperform gradient descent? Do all the other values of $\beta$ also outperform the gradient descent?

   *Take-home message*: hyperparameter tuning may be very time consuming, because you need to run the optimization algorithm (which may itself take a long time) for several different values of the hyperparameters. It is therefore tempting to carry out just a few optimization iterations (to save some time!): would any conclusion reached in such case be trustworthy?

In [ ]:
result_10 = pd.DataFrame(columns=[0.9,0.8,0.7,0.6], index=["result", "iterations", "cost"])
i = 0

for beta in [0.9, 0.8, 0.7, 0.6]:

  coef = {"alpha_hb": 10**(-2),
        "beta_hb": beta}

  all_w_hb, all_f_hb, all_grad_hb, k_hb = optimization(
    "heavy_ball",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 10,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)
  col = [0.9,0.8,0.7,0.6][i]
  if type(all_w_hb[-1, 0]) != Inf and type(all_w_hb[-1, 1]) != Inf :
    result_10.at["result", col] = "[" + "{:.4f}".format(all_w_hb[-1, 0]) + ", " + "{:.4f}".format(all_w_hb[-1, 1]) + "]"

  else:
    result_10.at["result", col] = "[" +str(all_w_hb[-1, 0])+ "]"

  result_10.at["iterations", col] = k_hb
  result_10.at["cost", col] = abs(all_f_hb[-1] - f_1([3, 0.5]))


  i=i+1

h = result_10.style.set_caption("HEAVY BALL: Final convergence result, iterations and cost function for different values of beta (Max iterations = 10)") \
.highlight_min(axis=1, subset= [0.6], props='color:black; background-color: #8deeee;')\
.format(na_rep="\\")

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
h.set_table_styles([headers, cells])


,0.900000,0.800000,0.700000,0.600000
result,"[215.9910, -764.1848]","[0.1492, -1.2403]","[1.0539, -0.8881]","[1.3258, -0.6252]"
iterations,10,10,10,10
cost,9291001164834934882304.000000,11.590894,5.043670,3.460150


This chart shows that none of the algorithms with different values of β reach the minimum point within 10 iterations. Among them, $\beta$ = 0.6 appears to be the best value, as it achieves the lowest cost function after 10 iterations.

In [ ]:
result_100 = pd.DataFrame(columns=[0.9,0.8,0.7,0.6], index=["result", "iterations", "cost"])
i = 0
for beta in [0.9, 0.8, 0.7, 0.6]:

  coef = {"alpha_hb": 10**(-2),
        "beta_hb": beta}

  all_w, all_f, all_grad, k = optimization(
    "heavy_ball",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)
  col = [0.9,0.8,0.7,0.6][i]
  if type(all_w[-1, 0]) != Inf and type(all_w[-1, 1]) != Inf :
    if all_w[-1, 0] > 10^100 or all_w[-1, 1] > 10^100: #in case of divergence
      result_100.at["result", col] = "[" + "{:.2e}".format(all_w[-1, 0]) + ", " + "{:.2e}".format(all_w[-1, 1]) + "]"
    else:
      result_100.at["result", col] = "[" + "{:.4f}".format(all_w[-1, 0]) + ", " + "{:.4f}".format(all_w[-1, 1]) + "]"

  else:
    result_100.at["result", col] = "[" +str(all_w[-1, 0])+ "]"

  result_100.at["iterations", col] = k
  result_100.at["cost", col] = abs(all_f[-1] - f_1([3, 0.5]))


  i=i+1

h1 = result_100.style.set_caption("HEAVY BALL: Final convergence result, iterations and cost function for different values of beta (Max iterations = 100)") \
.highlight_min(axis=1, subset= [0.8], props='color:black; background-color: #8deeee;')\
.format(na_rep="\\",
        formatter= {('0.9'): "{:.4e}"
                          })

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
h1.set_table_styles([headers, cells])

,0.900000,0.800000,0.700000,0.600000
result,"[2.59e+123, -9.17e+123]","[2.9486, 0.4869]","[2.8339, 0.4554]","[2.7516, 0.4308]"
iterations,12,100,100,100
cost,inf,0.000448,0.005347,0.013185


When considering the case with kmax = 100, the results differ. The algorithm with $\beta$ = 0.9 diverges, while the others converge. Among them, $\beta$ = 0.8 achieves the lowest cost function. This result suggests that 10 iterations are insufficient to determine the optimal value of $\beta$.

### Comparison with Gradient Descent

In [ ]:
all_w_G, all_f_G, all_grad_G,k_G = optimization(
    "gradient",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = {"alpha_gd": 10**(-2)})

all_w_HB, all_f_HB, all_grad_HB, k_HB = optimization(
    "heavy_ball",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = {"alpha_hb": 10**(-2),
        "beta_hb": 0.8})

fig = go.Figure()
all_methods = ["Gradient", "Heavy Ball"]
all_r_method = [all_f_G, all_f_HB]
for method_index in range(2):
    fig.add_scatter(
        x=np.arange(all_r_method[method_index].shape[0]), y=all_r_method[method_index],
        marker=dict(color=plotly.colors.qualitative.Set1[method_index], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[method_index], width=2),
        mode="lines+markers", name=all_methods[method_index] + " method"
    )
fig.update_layout(
    title="Cost Function - Gradient Method VS Heavy Ball method"
)
#fig.update_xaxes(range=[0, 15])
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

The heavy ball method showed a steeper decrease, achieving a lower cost function by the 100th iteration. The results obtained with both methods are summarized in the two charts below:

In [ ]:
g

,10^(0),10^(-1),10^(-2),10^(-3),10^(-4)
result,"[inf, inf]","[1.78e+243, 2.88e+243]","[2.4408, 0.3217]","[1.0486, 0.3166]","[0.6195, 1.1776]"
iterations,4,4,100,100,100
cost,\,inf,0.098490,4.911809,17.892408


In [ ]:
h1

,0.900000,0.800000,0.700000,0.600000
result,"[2.59e+123, -9.17e+123]","[2.9486, 0.4869]","[2.8339, 0.4554]","[2.7516, 0.4308]"
iterations,12,100,100,100
cost,inf,0.000448,0.005347,0.013185


Excluding the case where $\beta$ = 0.9, which does not converge, all other instances of the Heavy Ball method outperform the gradient method, achieving lower cost function values by the 100th iteration.

6. Repeat step 5 using Nesterov method instead of the heavy ball method. Answer to the same questions that were asked at step 5, and also compare heavy ball and Nesterov methods when using the respective best value for $\beta$.

In [ ]:
result_10 = pd.DataFrame(columns=[0.9,0.8,0.7,0.6], index=["result", "iterations", "cost"])
i = 0

for beta in [0.9, 0.8, 0.7, 0.6]:

  coef = {"alpha_n": 10**(-2),
        "beta_n": beta}

  all_w, all_f, all_grad, k = optimization(
    "nesterov",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 10,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)
  col = [0.9,0.8,0.7,0.6][i]
  if type(all_w[-1, 0]) != Inf and type(all_w[-1, 1]) != Inf :
    if all_w[-1, 0] > 10^100 or all_w[-1, 1] > 10^100:
      result_10.at["result", col] = "[" +  "{:.4e}".format(all_w[-1, 0]) + ", " +  "{:.4e}".format(all_w[-1, 1]) + "]"
    else:
      result_10.at["result", col] = "[" +  "{:.4f}".format(all_w[-1, 0]) + ", " +  "{:.4f}".format(all_w[-1, 1]) + "]"

  else:
    result_10.at["result", col] = "[" +str(all_w[-1, 0])+ "]"

  result_10.at["iterations", col] = k
  if abs(all_f[-1] - f_1([3, 0.5])) > 10^100:
    result_10.at["cost", col] = "{:.4e}".format(abs(all_f[-1] - f_1([3, 0.5])))
  else:
    result_10.at["cost", col] = "{:.4f}".format(abs(all_f[-1] - f_1([3, 0.5])))


  i=i+1

n = result_10.style.set_caption("NESTEROV: Final convergence result, iterations and cost function for different values of beta (Max iterations = 10)") \
.highlight_min(axis=1, subset= [0.6], props='color:black; background-color: #8deeee;')\
.format(na_rep="\\")

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
n.set_table_styles([headers, cells])

,0.900000,0.800000,0.700000,0.600000
result,"[-3.2878e+14, 2.9698e+14]","[0.5168, -1.3713]","[1.1614, -0.7863]","[1.3442, -0.4527]"
iterations,10,10,10,10
cost,7.4158e+115,7.9940,4.4025,2.9369


This chart shows that, within 10 iterations, none of the algorithms reach the minimum or a low cost function. The value of $\beta$ that appears to yield the lowest cost function is $\beta$ = 0.6.

In [ ]:
result_100 = pd.DataFrame(columns=[0.9,0.8,0.7,0.6], index=["result", "iterations", "cost"])
i = 0

for beta in [0.9, 0.8, 0.7, 0.6]:

  coef = {"alpha_n": 10**(-2),
        "beta_n": beta}

  all_w, all_f, all_grad, k = optimization(
    "nesterov",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)
  col = [0.9,0.8,0.7,0.6][i]
  if type(all_w[-1, 0]) != Inf and type(all_w[-1, 1]) != Inf :
    if all_w[-1, 0] > 10^100 or all_w[-1, 1] > 10^100:
      result_100.at["result", col] = "[" +  "{:.4e}".format(all_w[-1, 0]) + ", " +  "{:.4e}".format(all_w[-1, 1]) + "]"
    else:
      result_100.at["result", col] = "[" +  "{:.4f}".format(all_w[-1, 0]) + ", " +  "{:.4f}".format(all_w[-1, 1]) + "]"

  else:
    result_100.at["result", col] = "[" +str(all_w[-1, 0])+ "]"

  result_100.at["iterations", col] = k
  if abs(all_f[-1] - f_1([3, 0.5])) > 10^100:
    result_100.at["cost", col] = "{:.4e}".format(abs(all_f[-1] - f_1([3, 0.5])))
  else:
    result_100.at["cost", col] = abs(all_f[-1] - f_1([3, 0.5]))


  i=i+1

n_100 = result_100.style.set_caption("NESTEROV: Final convergence result, iterations and cost function for different values of beta (Max iterations = 100)") \
.highlight_min(axis=1, subset= [0.8], props='color:black; background-color: #8deeee;')\
.format(na_rep="\\")

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
n_100.set_table_styles([headers, cells])

,0.900000,0.800000,0.700000,0.600000
result,"[-inf, inf]","[2.9306, 0.4821]","[2.8249, 0.4528]","[2.7488, 0.4300]"
iterations,12,100,100,100
cost,\,0.000834,0.006004,0.013534


With 100 iterations, the algorithm does not converge for $\beta$ = 0.9, but achieves good results with $\beta$ = 0.7 and 0.8. The lowest cost function at the final iteration is obtained with $\beta$ = 0.8, which differs from the result at 10 iterations. This indicates that 10 iterations are insufficient to determine the optimal value for $\beta$."

### Comparison with Heavy Ball and Gradient Descent

The charts below display the final results obtained with the gradient descent method, the heavy ball method, and the Nesterov method after 100 iterations:

In [ ]:
g

,10^(0),10^(-1),10^(-2),10^(-3),10^(-4)
result,"[inf, inf]","[1.78e+243, 2.88e+243]","[2.4408, 0.3217]","[1.0486, 0.3166]","[0.6195, 1.1776]"
iterations,4,4,100,100,100
cost,\,inf,0.098490,4.911809,17.892408


In [ ]:
h1

,0.900000,0.800000,0.700000,0.600000
result,"[2.59e+123, -9.17e+123]","[2.9486, 0.4869]","[2.8339, 0.4554]","[2.7516, 0.4308]"
iterations,12,100,100,100
cost,inf,0.000448,0.005347,0.013185


In [ ]:
n_100

,0.900000,0.800000,0.700000,0.600000
result,"[-inf, inf]","[2.9306, 0.4821]","[2.8249, 0.4528]","[2.7488, 0.4300]"
iterations,12,100,100,100
cost,\,0.000834,0.006004,0.013534


As demonstrated earlier, the heavy ball method outperforms the gradient method and also achieves better results compared to the Nesterov method. However, it is important to note that the difference in cost function between these methods is minimal, with both methods nearly reaching the optimal point [3, 0.5]. These conclusions are also illustrated in the plot below.

In [ ]:
all_w_N, all_f_N, all_grad_N, k_N = optimization(
    "nesterov",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = {"alpha_n": 10**(-2),
        "beta_n": 0.8})

fig = go.Figure()
all_methods = ["Gradient", "Heavy Ball", "Nesterov"]
all_r_method = [all_f_G, all_f_HB, all_f_N]
for method_index in range(3):
    fig.add_scatter(
        x=np.arange(all_r_method[method_index].shape[0]), y=all_r_method[method_index],
        marker=dict(color=plotly.colors.qualitative.Set1[method_index], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[method_index], width=2),
        mode="lines+markers", name=all_methods[method_index] + " method"
    )
fig.update_layout(
    title="Error on the function value - different methods"
)
#fig.update_xaxes(range=[0, 15])
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

7. Run the adagrad method with initial point $\boldsymbol{w}_0 = (0.8, 1.6)$, tolerance $\varepsilon = 10^{-5}$,  $\alpha = 0.58$, regularization coefficient $\zeta = 10^{-8}$ and $K_{\max} = 100$. Compare the values of the cost function at the last iteration to the one you found using the best gradient method at step 4, which had $\alpha = 10^{-2}$. Why do the two methods perform so similarly, even tough there is a large difference between the two in the value of the hyperparameter $\alpha$?

   *Take-home message*: be careful when comparing hyperparameters in different algorithms, even when they are called with the same letter!

In [ ]:
comparison = pd.DataFrame(columns=["gradient", "adagrad"], index=["result", "iterations", "cost"])
i = 0
for METHOD in ["gradient", "adagrad"]:

  coef = {"alpha_gd": 0.01,
        "zeta_adag": 10**(-8),
        "alpha_adag": 0.58}

  all_w, all_f, all_grad, k = optimization(
    METHOD,
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)
  col = ["gradient", "adagrad"][i]

  comparison.at["result", col] = "[" +  "{:.4f}".format(all_w[-1, 0]) + ", " +  "{:.4f}".format(all_w[-1, 1]) + "]"
  comparison.at["iterations", col] = k
  comparison.at["cost", col] = abs(all_f[-1] - f_1([3, 0.5]))


  i=i+1

comp = comparison.style.set_caption("Gradient method VS Adagrad method: Final convergence result, iterations and cost function (Max iterations = 100)") \
.format(na_rep="\\")

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
comp.set_table_styles([headers, cells])

,gradient,adagrad
result,"[2.4408, 0.3217]","[2.4451, 0.3187]"
iterations,100,100
cost,0.098490,0.097438


This table shows that the final results of the two methods are very similar. Additionally, the plot below indicates that after approximately 33 iterations, the cost function curves of both methods nearly coincide. Despite the different values of $\alpha$  (`alpha_gd` = 0.01 and `alpha_adag` = 0.58), the methods perform similarly because the $\alpha$ values have different meanings. `alpha_gd` is a step length multiplied by the (negative) gradient at each iteration, whereas `alpha_adag` is multiplied by both the (negative) gradient and a scaling vector ${h_k}$.

In [ ]:
all_w_AD, all_f_AD, all_grad_AD, k_AD = optimization(
    "adagrad",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = {"zeta_adag": 10**(-8),
        "alpha_adag": 0.58})

fig = go.Figure()
all_methods = ["Gradient", "Adagrad"]
all_r_method = [all_f_G, all_f_AD]
for method_index in range(2):
    fig.add_scatter(
        x=np.arange(all_r_method[method_index].shape[0]), y=all_r_method[method_index],
        marker=dict(color=plotly.colors.qualitative.Set1[method_index], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[method_index], width=2),
        mode="lines+markers", name=all_methods[method_index] + " method"
    )
fig.update_layout(
    title="Error on the function value - different methods"
)
#fig.update_xaxes(range=[0, 15])
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

8. Run the adagrad method with initial point $\boldsymbol{w}_0 = (0.8, 1.6)$, tolerance $\varepsilon = 10^{-5}$,  $\alpha = 2.5$, regularization coefficient $\zeta = 10^{-8}$ and $K_{\max} = 100$.

   Change your adagrad implementation to print the entries of the vector $\alpha \boldsymbol{h}_k$, and check that its entries at $K_{\max} = 100$ are approximately $\alpha h_{100}^{(0)} = 0.04$ and $\alpha h_{100}^{(1)} = 0.026$. Run the gradient descent method with the following constant step lengths: $\alpha h_{100}^{(0)}$, $\alpha h_{100}^{(1)}$ and $(\alpha h_{100}^{(0)} + \alpha h_{100}^{(1)})/2$, and show that (again with $K_{\max} = 100$) they perform worse than adagrad.

   *Take-home message*: with the proper choice of $\alpha$, methods with diagonal scaling (with possibly different coefficients on the diagonal) outperform standard first order methods (i.e., methods in which all the coefficients on the diagonal are the same).

In [ ]:
def optimization_new(method, f, grad_f, epsilon, k_max, w_0, coef):
    assert method in ("gradient", "heavy_ball", "nesterov", "adagrad", "rmsprop", "adam")

    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]
    sHAT_k = []
    mHAT_k = []
    update = []

    # Prepare iteration counter
    k = 0

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]
        f_k = all_f[k]
        grad_f_k = all_grad_f[k]
        grad_f_k_beta = all_grad_f[k]


        # Define the search direction p_k
        if method == "gradient":
            #p_k_dot_g_k = - np.linalg.norm(grad_f_k)**2

            # NOT Carry out a backtracking line search
            alpha_k = coef.get("alpha_gd")

            # Compute w_{k+1}
            w_k_plus_1 = w_k - alpha_k * grad_f_k

        elif method == "heavy_ball":
            #p_k_dot_g_k = - np.linalg.norm(grad_f_k)**2

            # NOT Carry out a backtracking line search
            alpha_k = coef.get("alpha_hb")
            beta_k = coef.get("beta_hb")
            if k > 0:
              w_k_minus_1 = all_w[k-1]
            else:
              w_k_minus_1 = w_0
            # Compute w_{k+1}
            w_k_plus_1 = w_k - alpha_k * grad_f_k + beta_k * (w_k - w_k_minus_1)

        elif method == "nesterov":
         # p_k_dot_g_k = - np.linalg.norm(grad_f_k)**2

          alpha_k = coef.get("alpha_n")
          beta_k = coef.get("beta_n")
          if k > 0:
            w_k_minus_1 = all_w[k-1]
          else:
            w_k_minus_1 = w_0
            # Compute w_{k+1}
          z_k = w_k + beta_k*(w_k - w_k_minus_1)
          w_k_plus_1 = w_k - alpha_k * grad_f(z_k) + beta_k * (w_k - w_k_minus_1)


        elif method == "adagrad":
          alpha_k = coef.get("alpha_adag")
          zeta_k = coef.get("zeta_adag")
          s_k = sum(i*i for i in all_grad_f)
          h_k = 1/(np.sqrt(s_k)+zeta_k)
          update.append(alpha_k*h_k)

          w_k_plus_1 = w_k + update[k]*(-grad_f_k)




        elif method == "rmsprop":
          alpha_k = coef.get("alpha_RMS")
          zeta_k = coef.get("zeta_RMS")
          gamma_k = coef.get("gamma_RMS")
          if k > 1:
            sHAT_k_minus_1 = sHAT_k[k-1]
          else:
            sHAT_k_minus_1 = 0
          g_k = - grad_f_k
          sHAT_k.append((gamma_k * sHAT_k_minus_1) + ((1-gamma_k) * g_k) * g_k)
          update.append((alpha_k/(np.sqrt(np.array(sHAT_k[k])) + zeta_k)))
          w_k_plus_1 = w_k + update[k] * g_k


        elif method == "adam":
          gammaM_k = coef.get("gammaM_adam")
          gammaS_k = coef.get("gammaS_adam")
          alpha_k = coef.get("alpha_adam")
          zeta_k = coef.get("zeta_adam")
          g_k = - grad_f_k

          if k > 1:
            mHAT_k_minus_1 = mHAT_k[k-1]
          else:
            mHAT_k_minus_1 = 0

          mHAT_k.append((gammaM_k * mHAT_k_minus_1) + (1- gammaM_k)* g_k)


          if k > 1:
            sHAT_k_minus_1 = sHAT_k[k-1]
          else:
            sHAT_k_minus_1 = 0

          sHAT_k.append(gammaS_k* sHAT_k_minus_1 + ((1- gammaS_k)* g_k) * g_k)

          mTILDE_k = mHAT_k[k]/(1- (gammaM_k**(k+1)))
          sTILDE_k = sHAT_k[k]/(1- (gammaS_k**(k+1)))

          w_k_plus_1 = w_k + (alpha_k/(np.sqrt(np.array(sTILDE_k)) + zeta_k)) * mTILDE_k



        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))


        # Increment iteration counter
        k += 1

        if k >= k_max:
          #print("WARNING: " + method + " method terminated early due to reaching k_max = " + str(k))
          return np.array(all_w), np.array(all_f), np.array(all_grad_f), k, np.array(update)


    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f), k, np.array(update)

In [ ]:
coef = {"zeta_adag": 10**(-8),
        "alpha_adag": 2.5}

all_w, all_f, all_grad, k, update= optimization_new(
    "adagrad",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)

In [ ]:
ah_0 = update[-1,0]
ah_1 = update[-1,1]
print(ah_0, ah_1, (ah_0+ah_1)/2) #values of alpha to use in gradient method

0.040246226297001655 0.026072258577448933 0.033159242437225296


In [ ]:
comparison = pd.DataFrame(columns=["method", "alpha","result", "iterations", "cost"])

for i in range(4):
  method= ["adagrad","gradient","gradient","gradient"][i]
  alpha = 0
  if i > 0:
    alpha = [ah_0, ah_1, (ah_0+ah_1)/2][i-1]


  coef = {"alpha_gd": alpha,
        "zeta_adag": 10**(-8),
        "alpha_adag": 2.5}

  all_w, all_f, all_grad, k, update = optimization_new(
    method,
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)

  comparison.at[i, "method"] = method
  comparison.at[i, "alpha"] = alpha
  if abs(all_w[-1, 0]) > 10^100 or abs(all_w[-1, 1]) > 10^100:
    comparison.at[i, "result"] = "[" +  "{:.4e}".format(all_w[-1, 0]) + ", " +  "{:.4e}".format(all_w[-1, 1]) + "]"
  else:
    comparison.at[i, "result"] = "[" +  "{:.4f}".format(all_w[-1, 0]) + ", " +  "{:.4f}".format(all_w[-1, 1]) + "]"

  comparison.at[i, "iterations"] = k
  comparison.at[i, "cost"] = abs(all_f[-1] - f_1([3, 0.5]))



comp1 = comparison.style.set_caption("Gradient method VS Adagrad method: Alpha, final convergence result, iterations and cost function (Max iterations = 100)") \
.format(na_rep="\\", formatter= {('alpha'): "{:.4f}"
                          })

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
comp1.set_table_styles([headers, cells])

,method,alpha,result,iterations,cost
0,adagrad,0.0000,"[2.8570, 0.4618]",100,0.003859
1,gradient,0.0402,"[-4.4453e+208, -1.1433e+209]",5,inf
2,gradient,0.0261,"[2.7542, 0.4316]",100,0.012876
3,gradient,0.0332,"[2.8227, 0.4522]",100,0.006169


Comparing the results, it is evident that the Adagrad method achieves the lowest cost function value in 100 iterations. The gradient method does not converge with $\alpha_{gradient} = \alpha_{adagrad} h_{100}^{(0)}$, but it performs well with $\alpha_{gradient} = (\alpha h_{100}^{(0)} + \alpha h_{100}^{(1)})/2$. However, its cost function value remains slightly higher than that of the Adagrad method at the final iteration.



9. Call $\alpha^{\text{adagrad}}$ and $\alpha^{\text{RMSProp}}$ the respective scaling hyperparameters of the two methods. With the help of the formulae in the lecture, one could show that the formula (we will not be showing how to obtain this formula in this course)
$$\alpha^{\text{RMSProp}} = \alpha^{\text{adagrad}} \sqrt{1 - \gamma}$$
ensures that adagrad and RMSProp have the same diagonal scaling coefficients at the first iteration. This makes it possible to perform a fair comparison between the two methods, which will start to differ starting from the second iteration.

   Consider initial point $\boldsymbol{w}_0 = (0.8, 1.6)$, tolerance $\varepsilon = 10^{-5}$,  regularization coefficient $\zeta = 10^{-8}$, $\gamma = 0.9$, $K_{\max} = 100$.
   Run adagrad with $\alpha^{\text{adagrad}} = 0.25$, and RMSProp with $\alpha^{\text{RMSProp}} = \alpha^{\text{adagrad}} \sqrt{1 - \gamma} = 0.25 \sqrt{0.1}$. Show that
   * the entries of $\boldsymbol{h}_k^{\text{adagrad}}$ are monotically decreasing, and thus in particular $\boldsymbol{h}_{100}^{\text{adagrad}}$ will be smaller than $\boldsymbol{h}_{0}^{\text{adagrad}}$,
   * the entries of $\boldsymbol{h}_k^{\text{RMSProp}}$ are not monotonically decreasing, but are increasing at some iterations and decreasing at others. Furthermore, the entries at $\boldsymbol{h}_{100}^{\text{RMSProp}}$ are larger than those at $\boldsymbol{h}_0^{\text{RMSProp}}$.

   *Take-home message*: this is a concrete example on the adagrad limitation that we discussed during the lecture, i.e. vanishing diagonal scaling, which is one of the motivations to introduce RMSProp and adadelta.

In [ ]:
coef = {"zeta_adag": 10**(-8),
        "alpha_adag": 0.25,
        "gamma_RMS": 0.9,
        "alpha_RMS": 0.25*np.sqrt(0.1),
        "zeta_RMS": 10**(-8)}

all_w_adagrad, all_f_adagrad, all_grad_adagrad, k_adagrad, update_adagrad= optimization_new(
    "adagrad",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)

all_w_rmsprop, all_f_rmsprop, all_grad_rmsprop, k_rmsprop, update_rmsprop= optimization_new(
    "rmsprop",
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 100,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)


print(all_w_adagrad[-1,:], all_w_rmsprop[-1,:])

[1.92015326 0.13179113] [2.87578428 0.48807762]


In [ ]:
h_k_adagrad = update_adagrad/0.25
h_k_rmsprop =update_rmsprop/(0.25*np.sqrt(0.1))

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=np.array(range(101)), y=h_k_adagrad[:, 0], name = "adagrad: h_k[0]", marker_color='rgba(0,191,255, 1)'),
    row=1, col=1
)

fig.add_trace(
   go.Scatter(x=np.array(range(101)), y=h_k_adagrad[:, 1], name = "adagrad: h_k[1]", marker_color='rgba(28,134,238, 1)'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=np.array(range(100)), y=h_k_rmsprop[:, 0], name = "RMSprop: h_k[0]", marker_color='rgba(191,62,255,1 )'),
    row=1, col=1
)

fig.add_trace(
   go.Scatter(x=np.array(range(101)), y=h_k_rmsprop[:, 1], name = "RMSprop: h_k[1]", marker_color='rgba(148,0,211, 1)'),
    row=1, col=2
)

#fig.update_yaxes(range=[0, 0.1]) #to see better adagrad curve
fig.update_layout(height=600, width=800, title_text="Entries of h_k for Adagrad and RMS prop")
fig.show()

From these plots is possible to observe the trend of $h_k$ for both the methods:

In case of adagrad the curve for $h^{(0)}$ is equal to 0.02228159 at the beggining ($h_0^{(0)}$) and after 100 iterations $h_{100}^{(0)} = 0.012516034$ that is smaller. The curve has a secreasing trend and it is always lower to the one of the RMSprop method. $h^{(1)}$ is equal to 0.01193764 at the beggining ($h_0^{(1)}$) and after 100 iterations $h_{100}^{(1)} = 0.009274651$ that is also smaller to the inizial value, and also in this case the curve has a monotonically decreasing trend.

In case of RMSprop the curve for $h^{(0)}$ is equal to 0.07046057 at the beggining $[(h_0^{(0)})_{RMSprop} < (h_0^{(0)})_{adagrad}]$. Then, it doesn't have a monotone trend, it has mainly 3 peaks, but at the end the value of $h^{(0)}$ is higher than the beggining: $h_{100}^{(0)} = 1.734536$. The curve for $h^{(1)}$ has mainly 2 peaks: at the first iteration $h_0^{(1)} = 0.03775012$, it has some ups and down and at the final iteration it reaches an higher value than the starting one $h_{100}^{(1)}=0.496386$.  


10. Call $\alpha^{\text{RMSProp}}$ and $\alpha^{\text{adam}}$ the respective scaling hyperparameters of the two methods. With the help of the formulae in the lecture, one could show that the formula (we will not be showing how to obtain this formula in this course)
$$\alpha^{\text{RMSProp}} = \alpha^{\text{adam}} \sqrt{1 - \gamma_s}$$
ensures that RMSProp and Adam have the same diagonal scaling coefficients at the first iteration. This makes it possible to perform a fair comparison between the two methods, which will start to differ starting from the second iteration.

   Consider initial point $\boldsymbol{w}_0 = (0.8, 1.6)$, tolerance $\varepsilon = 10^{-5}$, regularization coefficient $\zeta = 10^{-8}$.
   Furthermore, choose RMSProp decay coefficient as $\gamma = 0.999$, and Adam decay coefficients as $\gamma_m = 0.9$ and $\gamma_s = 0.999$. Note that the decay $\gamma$ in the RMSProp plays the same role of the decay $\gamma_s$ in Adam method, therefore we choose the same value. Set $\alpha^{\text{adam}} = 0.25$ and $\alpha^{\text{RMSProp}} = \alpha^{\text{adam}} \sqrt{1 - \gamma_s} = 0.25 \sqrt{0.001}$, and a value $K_{\max} = 10000$, larger than what we have used until now. Show that Adam converges to the prescribed tolerance in a number of iterations which is approximately $\frac{1}{8}$ of the number of iterations required by RMSProp.

   *Take-home message*: you have experimented for the first time in this course with Adam, a state of the art optimization method that is very used in machine learning, and seen that it outperforms other related methods.

In [ ]:
comparison = pd.DataFrame(columns=["method", "result", "iterations", "cost"])
fig = go.Figure()
all_methods = ["rmsprop", "adam"]


for i in range(2):
  method= all_methods[i]
  alpha = 0

  coef = {"gamma_RMS": 0.999,
        "alpha_RMS": 0.25*np.sqrt(0.001),
        "zeta_RMS": 10**(-8),
        "gammaM_adam": 0.9,
        "gammaS_adam": 0.999,
        "alpha_adam": 0.25,
        "zeta_adam": 10**(-8)}

  all_w, all_f, all_grad, k = optimization(
    method,
    f_1,
    grad_f_1,
    epsilon = 10**(-5),
    k_max = 10000,
    w_0 = np.array([0.8, 1.6]),
    coef = coef)

  comparison.at[i, "method"] = method
  comparison.at[i, "result"] = "[" + "{:.6f}".format(all_w[-1, 0]) + ", " + "{:.6f}".format(all_w[-1, 1]) + "]"
  comparison.at[i, "iterations"] = k
  comparison.at[i, "cost"] = abs(all_f[-1] - f_1([3, 0.5]))

  if i == 0:
    all_f_RMS = all_f
    all_f_ADAM = 0
  else:
    all_f_ADAM = all_f

  all_r_method = [all_f_RMS, all_f_ADAM]

  fig.add_scatter(
        x=np.arange(all_r_method[i].shape[0]), y=all_r_method[i],
        marker=dict(color=plotly.colors.qualitative.Set1[i], size=10),
        line=dict(color=plotly.colors.qualitative.Set1[i], width=2),
        mode="lines+markers", name=all_methods[i] + " method"
    )



comp2 = comparison.style.set_caption("RMSProp VS Adam: final convergence result, iterations and cost function (Max iterations = 1000)") \
.format(na_rep="\\")

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #009acd; color: white;text-align: center;font-weight:bold;'
}
cells = {'selector': 'td',
         'props': 'text-align: center;'}
comp2.set_table_styles([headers, cells])

,method,result,iterations,cost
0,rmsprop,"[2.999968, 0.499992]",2297,0.000000
1,adam,"[3.000014, 0.500003]",242,0.000000


The chart shows that both methods converge to the minimum, but they do so with different numbers of iterations. RMSprop takes 2297 iterations to converge, whereas the Adam method converges in just 242 iterations. The plot below illustrates that the Adam method requires fewer than 1/8 of the iterations needed by RMSprop to achieve convergence.

In [ ]:
fig.add_vline(x=(1/8) * 2297, line_dash="dot",
              annotation_text="1/8 iterations required by RMSprop",
              annotation_position="top right")
fig.update_layout(
    title="Error on the function value - different methods"
)

fig.update_yaxes(type="log", exponentformat="power")
fig.show()